<a href="https://colab.research.google.com/github/DonRomaniello/CitibikeDocks/blob/master/Monthly_CSV_Creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import time

import os
import tarfile

import json

In [2]:
#Set initial directory as the directory where the script is running.

initial_directory = '.'
os.chdir(initial_directory)
print("Current working directory is", os.getcwd())

Current working directory is /Users/donromaniello


In [ ]:
# Select directories

directory_prompt_selection = 'none'

while directory_prompt_selection != 'w':
  current_directories = [d for d in os.listdir('.') if os.path.isdir(d)]

  print("Current working directory is", os.getcwd())

  for directory in current_directories:

    print(current_directories.index(directory), directory)

  print("w", "We can use current directory")
  print("c", "Change top level directory")
  print("u", "Navigate up one level.")
  directory_prompt_selection = input("Choose an option:")
  print("\n")
  
  if directory_prompt_selection == 'c':
    directory_prompt_selection = input('Provide either the relative or full path name of the directory')
    os.chdir(directory_prompt_selection)
  elif directory_prompt_selection == 'u':
    os.chdir('../')
  elif directory_prompt_selection == 'w':
    None
  else:
    os.chdir(os.path.join('.', current_directories[int(directory_prompt_selection)]))

print("Current working directory is", os.getcwd())

In [ ]:
# Get rid of anything the operating system may add...
files = ([x for x in sorted(filter(os.path.isfile,
                                   os.listdir('.')),
                                   key=os.path.getmtime) if x[-2:] == "xz"])

files

In [ ]:
stopwatch_start = time.time()
stopwatch_hour = time.time()

stations_big = pd.DataFrame()


files_by_month = [x for x in sorted(filter(os.path.isfile,
                                          os.listdir('.')),
                                    key=os.path.getmtime) if x[-2:] == "xz"]


hour = tarfile.open('./' + files_by_month[0])
hour_list = hour.getnames()

minute = 0
minute_json = json.load(hour.extractfile(hour_list[minute]))
timestamp = minute_json["last_updated"]
stations_tick = (pd.DataFrame(minute_json['data']['stations'])
                [['station_id', 'num_docks_available']]
                .rename(columns={'num_docks_available':timestamp})
                .astype('int16')
                .set_index('station_id')
                .sort_index()[timestamp])

minute+=1

minute_json = json.load(hour.extractfile(hour_list[minute]))
timestamp = minute_json["last_updated"]
stations_tock = (pd.DataFrame(minute_json['data']['stations'])
                [['station_id', 'num_docks_available']]
                .rename(columns={'num_docks_available':timestamp})
                .astype('int16')
                .set_index('station_id')
                .sort_index()[timestamp])

stations_tick = pd.concat([stations_tick, stations_tock], join="outer", axis=1)

minute+=1
while minute < len(hour_list):
  try:
    minute_json = json.load(hour.extractfile(hour_list[minute]))
    timestamp = minute_json["last_updated"]

    stations_tock = (pd.DataFrame(minute_json['data']['stations'])
              [['station_id', 'num_docks_available']]
              .rename(columns={'num_docks_available':timestamp})
              .astype('int16')
              .set_index('station_id')
              .sort_index()[timestamp])

    if stations_tick.index.equals(stations_tock.index):
      stations_tick.insert((stations_tick.shape[1]), timestamp, stations_tock)
    else:
      stations_tick = pd.concat([stations_tick,stations_tock],
                            join='outer',
                            axis=1)

    minute+=1
  except json.JSONDecodeError:
    minute+=1
    print(stations_tick.shape)    

print(time.time() - stopwatch_hour)
stopwatch_hour = time.time()

for hour_xz in files_by_month[1:]:
  hour = tarfile.open('./' + hour_xz)
  hour_list = hour.getnames()
  minute=0

  print(stations_tick.shape)    
  print(time.time() - stopwatch_hour)
  stopwatch_hour = time.time()

  while minute < len(hour_list):
    try:
      minute_json = json.load(hour.extractfile(hour_list[minute]))
      timestamp = minute_json["last_updated"]

      stations_tock = (pd.DataFrame(minute_json['data']['stations'])
                [['station_id', 'num_docks_available']]
                .rename(columns={'num_docks_available':timestamp})
                .astype('int16')
                .set_index('station_id')
                .sort_index()[timestamp])

      if stations_tick.index.equals(stations_tock.index):
        stations_tick.insert((stations_tick.shape[1]), timestamp, stations_tock)
      else:
        stations_tick = pd.concat([stations_tick,stations_tock],
                              join='outer',
                              axis=1)

      if stations_tick.shape[1] > 1440:
        stations_big = pd.concat([stations_big, stations_tick], join='outer', axis=1)
        stations_tick = pd.DataFrame(index=stations_tick.index)
      else:
        pass

      minute+=1
    except json.JSONDecodeError:
      minute+=1



stopwatch_end = time.time()
print(stopwatch_end - stopwatch_start)

In [116]:
stations_big.to_csv(('./' + os.getcwd().split('/')[-1] + 'csv.gzip'))